# Maxim SDK tracing using Langchain Decorators

In [1]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv

# Load environment variables at the beginning of the notebook
load_dotenv()

maxim_api_key = os.environ.get("MAXIM_API_KEY", "")
log_repo_id = os.environ.get("MAXIM_LOG_REPO_ID", "")
openai_key = os.environ.get("OPENAI_API_KEY", "")
weaviate_url = os.environ.get("WEAVIATE_URL", "")
weaviate_key = os.environ.get("WEAVIATE_API_KEY", "")

In [2]:
from maxim import Config, Maxim
from maxim.logger import LoggerConfig
from maxim.decorators.langchain import langchain_callback, langchain_llm_call


logger = Maxim(Config()).logger(LoggerConfig(id=log_repo_id))

In [3]:
import json
from maxim.decorators import current_retrieval, current_trace, retrieval, trace
from langchain.chat_models.openai import ChatOpenAI
from langchain.tools import tool
from langchain_community.vectorstores import Weaviate

llm = ChatOpenAI(api_key=openai_key, model="gpt-4o-mini")


@retrieval(name="weaviate-call")
def retrieve_docs(query: str):    
    current_retrieval().input(query)
    response_objects = [
        {
            "properties": {
                "title": "Movie Database Entry 1",
                "year": 2022,
                "description": "Sample movie description",
                "genres": ["Action"],
                "actors": ["Actor 1", "Actor 2"],
                "director": "Director Name"
            }
        },
        {
            "properties": {
                "title": "Movie Database Entry 2",
                "year": 2021,
                "description": "Sample movie description",
                "genres": ["Drama"],
                "actors": ["Actor 3", "Actor 4"],
                "director": "Director Name"
            }
        },
        {
            "properties": {
                "title": "Movie Database Entry 3",
                "year": 2020,
                "description": "Sample movie description",
                "genres": ["Comedy"],
                "actors": ["Actor 5", "Actor 6"],
                "director": "Director Name"
            }
        }
    ]
    return response_objects


@langchain_llm_call(name="llm-call")
def execute(query: str):
    context = retrieve_docs(query)
    messages = [
        (
            "system",
            f"You answer questions about movies. Use provided list of movies to refine the response.\n\n List of movies: {json.dumps(context)}\n Respond in proper markdown format",
        ),
        ("human", query),
    ]
    result = llm.invoke(messages, config={"callbacks": [langchain_callback()]})
    return result.content


@trace(logger=logger, name="movie-chat-v1")
def generate(query: str):
    return execute(query)
    

query = "What is the capital of France?"
result = generate(query)
print(result)

/var/folders/p0/3mgzlyg541g1j9s2d1wskc040000gn/T/ipykernel_87080/622684663.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(api_key=openai_key, model="gpt-4o-mini")


The capital of France is Paris. However, if you have any questions about movies, feel free to ask!
